In [15]:
from bs4 import BeautifulSoup
import os
import json
from urllib import parse

def readXml(filepath):
    return BeautifulSoup(open(filepath,encoding="utf8"))

def traceChildren(children):
    for child in children:
        print("~~~",child)
def traceObj(obj):
    print("traceObj:",obj)
    for key in obj:
        print(key)

def adptUrl(url):
    if not url:
        return url
    if "orzooo.com" in url and "?n=" in url:
        url=url.split("?n=")[1]
        #print(url)
        url="@Page:"+parse.unquote(url)
        #print(url)
        return url
    return url
def getNodeClass(node):
    try:
        return node["class"]
    except:
        return None;

def parseLeftUL(ul,childList):
    #print("parseLeftUL:",ul)
    #traceChildren(ul.children)
    
    for child in ul.children:
        aO=child.find("a")
        if aO==None:
            print(child.string)
            aObj={}
            aObj["title"]=child.string.strip()
            childList.append(aObj)
            continue
        #print("aO:",aO)
        aObj={};
        aObj["href"]=adptUrl(aO["href"]);
        aObj["title"]=aO.string
        childList.append(aObj)
    
def parseWikiLeft(leftPart):
    #traceChildren(leftPart.children)
    navList=[]
    for child in leftPart.children:
        #print("!!!",type(child),child.name)
        tChildClass=getNodeClass(child)
        if child.name=="ul":
            if tNavO:
                parseLeftUL(child,tNavO["child"])
            
            continue
            
        if tChildClass!=None:
            if "sidehead" in tChildClass:
                #print(child)
                if child.a:
                    break
                tNavO={};
                tNavO["title"]=child.string
                if tNavO["title"]!=None:
                    tNavO["title"]=tNavO["title"].strip()
                tNavO["child"]=[]
                navList.append(tNavO)
        #traceObj(child)
    #print(navList)
    return navList

def parseWikiText(textPart):
    navList=[]
    tNavO=None
    for child in textPart.children:
        #print("!!!",type(child),child.name)
        tChildClass=getNodeClass(child)
        if child.name=="ul":
            if tNavO:
                parseLeftUL(child,tNavO["child"])
            else:
                tNavO={};
                tNavO["title"]="Content"
                tNavO["child"]=[]
                navList.append(tNavO)
                parseLeftUL(child,tNavO["child"])
            continue
            
        if tChildClass!=None:
            if "vspace" in tChildClass:
                #print(child)
                tNavO={};
                tNavO["title"]=child.string
                if tNavO["title"]!=None:
                    tNavO["title"]=tNavO["title"].strip()
                tNavO["child"]=[]
                navList.append(tNavO)
        else:
            if tNavO==None:
                if child.name=="p" and child.string.strip()!="":
                    tNavO={};
                    tNavO["title"]=child.string
                    if tNavO["title"]!=None:
                        tNavO["title"]=tNavO["title"].strip()
                    tNavO["child"]=[]
                    navList.append(tNavO)
                    
        #traceObj(child)
    #print("wikitext:",navList)
    return navList

def parseWikiTitle(titlePart):
    group=titlePart.find(None,"pagegroup")
    print("group:",group.a.string)
    title=titlePart.find(None,"pagetitle")
    print("title:",title.string.strip())
    infoO={};
    infoO["group"]=group.a.string.strip()
    infoO["title"]=title.string.strip()
    return infoO
def saveNavObj(navlist):
    navO={};
    navO["Nav"]=navlist
    fw=open("website/"+"nav"+".json","w",encoding="utf-8")
    json.dump(navO,fw,ensure_ascii=False,indent=4)
def getStructDataFromHtmlO(xmlO):
    rstO={}
    leftPart=xmlO.find(id="wikileft")
    
    
    #rstO["Nav"]=parseWikiLeft(leftPart)
    saveNavObj(parseWikiLeft(leftPart))

    wikiTitle=xmlO.find(id="wikititle")
    #print(wikiTitle)
    rstO["Info"]=parseWikiTitle(wikiTitle)

    wikiText=xmlO.find(id="wikitext")
    #print(wikiText)
    rstO["Content"]=parseWikiText(wikiText)
    
    return rstO
 
def workHtmPage(patePath,fileName):
    xmlO=readXml(patePath)
    dataO=getStructDataFromHtmlO(xmlO)
    fw=open("website/"+fileName+".json","w",encoding="utf-8")
    json.dump(dataO,fw,ensure_ascii=False,indent=4)

def getDirFiles(dirPath):
    files=os.listdir(dirPath)
    print(files)
    for tfile in files:
        tfile=dirPath+"/"+tfile
        #print("isdir:",os.path.isdir(tfile))
        if os.path.isdir(tfile):
            continue
        #print(tfile)
        bname=os.path.basename(os.path.realpath(tfile))
       # print(bname)
        names=bname.split(".")
        names.pop()
        names=names[0].split(" _ ")
        names.pop(0)
        #print(names)
        #print("_".join(names))
        workHtmPage(tfile,"_".join(names))
#workHtmPage("wiki/OrzWiki _ 学习 _ 量化交易.htm")

getDirFiles("wiki")


['OrzWiki _ Main _ HomePage.htm', 'OrzWiki _ Main _ HomePage_files', 'OrzWiki _ Other _ Ideas.htm', 'OrzWiki _ Other _ Ideas_files', 'OrzWiki _ SNS _ 推荐引擎.htm', 'OrzWiki _ SNS _ 推荐引擎_files', 'OrzWiki _ SNS _ 网站相关.htm', 'OrzWiki _ SNS _ 网站相关_files', 'OrzWiki _ 学习 _ AR.htm', 'OrzWiki _ 学习 _ AR_files', 'OrzWiki _ 学习 _ As3.htm', 'OrzWiki _ 学习 _ As3_files', 'OrzWiki _ 学习 _ IDE.htm', 'OrzWiki _ 学习 _ IDE_files', 'OrzWiki _ 学习 _ Js.html', 'OrzWiki _ 学习 _ Js_files', 'OrzWiki _ 学习 _ Python.htm', 'OrzWiki _ 学习 _ Python_files', 'OrzWiki _ 学习 _ TodoList.htm', 'OrzWiki _ 学习 _ TodoList_files', 'OrzWiki _ 学习 _ 产品Amp运营.htm', 'OrzWiki _ 学习 _ 产品Amp运营_files', 'OrzWiki _ 学习 _ 公开课forDear.htm', 'OrzWiki _ 学习 _ 公开课forDear_files', 'OrzWiki _ 学习 _ 动画相关.htm', 'OrzWiki _ 学习 _ 动画相关_files', 'OrzWiki _ 学习 _ 安全相关.htm', 'OrzWiki _ 学习 _ 安全相关_files', 'OrzWiki _ 学习 _ 开放平台们.htm', 'OrzWiki _ 学习 _ 开放平台们_files', 'OrzWiki _ 学习 _ 数据分析.htm', 'OrzWiki _ 学习 _ 数据分析_files', 'OrzWiki _ 学习 _ 算法.htm', 'OrzWiki _ 学习 _ 算法_files', 'OrzWi

D:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file D:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


%e7%94%9f%e6%b4%bb.%e7%94%b5%e5%bd%b1
@Page:生活.电影
%e7%94%9f%e6%b4%bb.%e8%bf%bd%e5%89%a7
@Page:生活.追剧
%e7%94%9f%e6%b4%bb.%e5%8a%a8%e6%bc%ab
@Page:生活.动漫
%e7%94%9f%e6%b4%bb.%e8%af%bb%e4%b9%a6
@Page:生活.读书
%e7%94%9f%e6%b4%bb.%e6%b8%b8%e6%88%8f
@Page:生活.游戏
%e7%94%9f%e6%b4%bb.%e9%9f%b3%e4%b9%90
@Page:生活.音乐
%e7%94%9f%e6%b4%bb.%e5%bf%83%e7%90%86%e7%a4%be%e4%bc%9a%e6%98%9f%e5%ba%a7
@Page:生活.心理社会星座
%e7%94%9f%e6%b4%bb.%e5%9f%9f%e5%90%8d
@Page:生活.域名
%e5%b7%a5%e5%85%b7.%e5%af%bc%e8%88%aa%e5%af%bc%e8%88%aa
@Page:工具.导航导航
%e5%b7%a5%e5%85%b7.Chrome
@Page:工具.Chrome
Site.StockLab
@Page:Site.StockLab
%e5%ad%a6%e4%b9%a0.%e8%af%ad%e6%b3%95%e5%88%86%e6%9e%90
@Page:学习.语法分析
%e5%ad%a6%e4%b9%a0.Python
@Page:学习.Python
%e5%ad%a6%e4%b9%a0.As3
@Page:学习.As3
%e5%ad%a6%e4%b9%a0.Js
@Page:学习.Js
%e5%ad%a6%e4%b9%a0.AR
@Page:学习.AR
%e5%ad%a6%e4%b9%a0.%e6%95%b0%e6%8d%ae%e5%88%86%e6%9e%90
@Page:学习.数据分析
%e5%ad%a6%e4%b9%a0.%e9%87%8f%e5%8c%96%e4%ba%a4%e6%98%93
@Page:学习.量化交易
%e5%ad%a6%e4%b9%a0.%e8%b0%83%e8%af%95%e5%b7%a5%e5%85%b7
@P

In [14]:
from urllib import parse
def adptUrl(url):
    if not url:
        return url
    if "orzooo.com" in url and "?n=" in url:
        url=url.split("?n=")[1]
        print(url)
        url="@Page:"+parse.unquote(url)
        print(url)
        return url
    return url
urlpath="http://orzooo.com/index.php?n=%e7%94%9f%e6%b4%bb.%e8%bf%bd%e5%89%a7"
print("orzooo.com" in urlpath)
adptUrl(urlpath)
adptUrl(None)

True
%e7%94%9f%e6%b4%bb.%e8%bf%bd%e5%89%a7
@Page:生活.追剧
